<a href="https://colab.research.google.com/github/ismachy/Introduction-to-ML/blob/main/HW_7_Problem_2_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import time

In [ ]:
# Load and preprocess CIFAR-10 data
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

170498071/170498071 [==============================] - 11s 0us/step


In [ ]:
train_images, test_images = train_images / 255.0, test_images / 255.0
train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

In [ ]:
def identity_block(x, filters):
    f1, f2, f3 = filters

    x_shortcut = x

    x = layers.Conv2D(f1, (1, 1), padding='valid')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(f2, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(f3, (1, 1), padding='valid')(x)
    x = layers.BatchNormalization()(x)

    x_shortcut = layers.Conv2D(f3, (1, 1), padding='valid')(x_shortcut)
    x_shortcut = layers.BatchNormalization()(x_shortcut)

    x = layers.add([x, x_shortcut])
    x = layers.Activation('relu')(x)

    return x


In [ ]:
# ResNet-10 model
input_shape = (32, 32, 3)
inputs = tf.keras.Input(shape=input_shape)

x = layers.Conv2D(16, (3, 3), padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

for _ in range(3):  # Number of blocks in ResNet-10
    x = identity_block(x, filters=[16, 16, 64])

for _ in range(3):
    x = identity_block(x, filters=[32, 32, 128])

for _ in range(3):
    x = identity_block(x, filters=[64, 64, 256])

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(10, activation='softmax')(x)

In [ ]:
model = models.Model(inputs, x)

# model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'],
              run_eagerly=True)

In [ ]:
# Train the model
start_time = time.time()
output = model.fit(train_images, train_labels, epochs=300, validation_data=(test_images, test_labels))
end_time = time.time()

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 23s 74ms/step - loss: 1.6431 - accuracy: 0.5197


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

# Predict on the test set
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(test_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='macro')

# Report results
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print(f"F1 Score: {f1}")

313/313 [==============================] - 20s 62ms/step
Accuracy: 0.5197
Confusion Matrix:
[[258  37  54   3  18   0   3  18 591  18]
 [  2 590   3   1   3   0   1   2 323  75]
 [ 38  13 545  11 152   4  37  30 166   4]
 [ 16  15 161 277 154  27  41  50 241  18]
 [  2   2 125   6 682   2  26  43 110   2]
 [  1   9 125 184 227 187  20 122 116   9]
 [  4   4 139  31 104   0 498   5 214   1]
 [  1   1  66  13 199   2   2 584 123   9]
 [  4   3   6   0   2   0   1   0 984   0]
 [  3  25   1   1   9   0   0   6 363 592]]
F1 Score: 0.5166134659575403


In [ ]:
# Report results
print(f"Training Time: {end_time - start_time} seconds")
print(f"Training Loss after 300 epochs: {output.history['loss'][-1]}")
print(f"Evaluation Accuracy after 300 epochs: {test_acc}")

Training Time: 26473.470908164978 seconds
Training Loss after 300 epochs: 0.9981153011322021
Evaluation Accuracy after 300 epochs: 0.5196999907493591


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 32, 32, 16)           448       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 32, 32, 16)           64        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 32, 32, 16)           0         ['batch_normalization[0][0